In [1]:
import pandas as pd

In [ ]:
dataset_name = 'amonia'
raiz = '../repositorio'

### Prata

In [71]:
# Abas das tabelas na planilha "Emissões de amonia.xlsx"
tabelas = pd.ExcelFile(f'{raiz}/Emissões de amonia.xlsx').sheet_names

anos = (pd.read_excel(f'{raiz}/Emissões de amonia.xlsx',
                sheet_name='NH3 Fertilizante', # Tabela dentro do arquivo de planilha
                header=2, # Linha do cabeçalho da tabela
                nrows=0, # Ignorar todas as linhas, utilizando apenas o cabeçalho
                usecols=lambda x: isinstance(x, int) # Função para listar apenas colunas numéricas 
                )
        .columns
        .tolist()
        )

In [117]:
df = [pd.concat([pd.DataFrame(), # Dataframe vazio como base para ser concatenado
                 (pd.read_excel(f'{raiz}/Emissões de amonia.xlsx', # Arquivo com resultados definitivos
                sheet_name=NH3,
                header=2,
                skiprows=lambda linha: linha in [3, 31, 32] # Linhas da tabela a serem ignoradas, se sobrepondo ao header
                )
                .merge(pd.read_parquet(f'{raiz}/prata_geocodigo_ibge.parquet') # Banco de dados com geocódigos
                        .filter(['UF-id', 'UF-nome'])
                        .drop_duplicates(), # Exclusão de duplicatas (geradas por ser um BD com municípios)
                        how='left',
                        left_on='Estado',
                        right_on='UF-nome'
                            )
                # Derretimento da tabela, verticalizando-a
                .melt(
                    id_vars='UF-id', # Coluna a ser mantida
                    value_vars=anos, # Colunas a serem transformadas em linhas a partir da lista criada anteriormente
                    var_name='ano', # Nome da nova coluna com os dados transformados em linhas
                    value_name='gas_ton' # Nome da nova coluna com os valores das antigas colunas
                            )
                #.query('gas_ton != 0.0') # Exclusão de registros que possuam valor zerado
                .rename(columns={'UF-id':'uf'})
                .assign(**{
                    'tipo': NH3[4:].capitalize(), # Criação de nova coluna com os títulos das tabelas cortando 'NH3' 
                    'fonte': 'NH3 volatizado' # Criação de nova coluna da categoria enquadrada
                        })
                .convert_dtypes()
                .sort_values(by=['uf', 'ano']) # Organização dos dados
                .set_index(['uf', 'ano', 'tipo'])
                )
    ]) for NH3 in tabelas]
# Resultado gerado a partir da compreensão de lista (list compreenshion)

### Ouro

In [123]:
(pd.DataFrame()
 .append(df) # Transformação de todas as listas em um dataframe ao fundir com um vazio
)

/tmp/ipykernel_619/1939853408.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  .append(df)
